In [156]:
using GradientRobustMultiPhysics
using ExtendableGrids
using ExtendableSparse
using GridVisualize
using SimplexGridFactory
using Triangulate
using LinearAlgebra
using Plots
include("../src/flowdata.jl")
include("../src/variationaltimedisc.jl")

Main.VariationalTimeDisc

In [161]:
function run_dg_navier_stokes(problem=1, order=2, Re=1e0, level=2, time_order = 1, scheme = 0)
    @assert order in 1:4
    
    ## FEType Pk + enrichment + pressure    
    FETypes = [H1Pk{2,2,order}, H1Pk{1,2,order-1}]
    
    # set the time discretization
    ## choose the time discretization 
    TDisc = VariationalTimeDisc.SetVariationalTimeDisc(Float64, time_order, scheme)  
    ## get exact flow data
    points,u,dtu,p,∇u,f,μ,periodic = get_problemdata(problem, Re = Re)
    
    # grid and refinement
    Solution = nothing
    xgrid = nothing
    pvis = GridVisualizer(; Plotter = nothing, layout = (1,2), clear = true, resolution = (1000,500))
    xgrid = get_grid(level; points = points, uniform = true)
    #@show level xgrid
    Problem = PDEDescription("Navier-Stokes problem")
    add_unknown!(Problem; equation_name = "momentum equation (Pk part)", unknown_name = "u_Pk")
    add_unknown!(Problem; equation_name = "incompressibility constraint", unknown_name = "p",
        algebraic_constraint = true)
    
    FES = [FESpace{FETypes[1]}(xgrid), FESpace{FETypes[2]}(xgrid; broken = true)]
    Solution = FEVector(FES)
    @show Problem
    
    # assembling matrices at time = 0
    M = FEMatrix{Float64}(FES)
    assemble_operator!(M[1,1], BilinearForm([Identity, Identity]))
    # velocity-velocity and pressure-velocity blocks
    A = FEMatrix{Float64}(FES)
    assemble_operator!(A[1,1], LaplaceOperator(μ))
    assemble_operator!(A[1,2], LagrangeMultiplier(Divergence); At = A[2,1]) 
    # right hand side
    rhs = FEVector{Float64}(FES)
    assemble_operator!(rhs[1], LinearForm(Identity, u); time=0.0)
    
end

run_dg_navier_stokes (generic function with 7 methods)

In [162]:
run_dg_navier_stokes()


VariationalTimeDisc(r=1, k=0): dG(1)
"Linear space and time example!!!!" = "Linear space and time example!!!!"
Problem = 
PDE-DESCRIPTION
  system name = Navier-Stokes problem

      id    | unknown name / variables [#A, #T] / equation name
     [1]    | u_Pk / ["u", "v"] / momentum equation (Pk part) 
     [2]    | p / ["p", "q"] / incompressibility constraint 

  LHS block | PDEOperator(s)
    [1,1]   | none
    [1,2]   | none
    [2,1]   | none
    [2,2]   | none

  RHS block | PDEOperator(s)
     [1]    | none
     [2]    | none

   BoundaryOperators[1] : 
   BoundaryOperators[2] : 




[ Info: Setting up uniform grid 
